In [16]:
import requests
from lxml import html
import re

Search for articles in arXiv

In [17]:
response = requests.get('https://arxiv.org/search/cs?query=deep+learning&searchtype=all&abstracts=show&order=-announced_date_first&size=25')
response

<Response [200]>

Parse response with lxml parser

In [18]:
doc_tree = html.fromstring(response.content)
doc_tree

<Element html at 0x10fe24278>

In [19]:
doc_tree.xpath('//*[@id="main-container"]//li[@class="arxiv-result"]/div/p/a')

[<Element a at 0x10fe245e8>,
 <Element a at 0x10fe24908>,
 <Element a at 0x10fe244a8>,
 <Element a at 0x10fe249a8>,
 <Element a at 0x10fe249f8>,
 <Element a at 0x10fe24a48>,
 <Element a at 0x10fe24a98>,
 <Element a at 0x10fe24ae8>,
 <Element a at 0x10fe24b38>,
 <Element a at 0x10fe24b88>,
 <Element a at 0x10fe24bd8>,
 <Element a at 0x10fe24c28>,
 <Element a at 0x10fe24c78>,
 <Element a at 0x10fe24cc8>,
 <Element a at 0x10fe24d18>,
 <Element a at 0x10fe24d68>,
 <Element a at 0x10fe24db8>,
 <Element a at 0x10fe24e08>,
 <Element a at 0x10fe24e58>,
 <Element a at 0x10fe24ea8>,
 <Element a at 0x10fe24ef8>,
 <Element a at 0x10fe24f48>,
 <Element a at 0x10fe24f98>,
 <Element a at 0x10fe2c048>,
 <Element a at 0x10fe2c098>]

Search for all links to articles.
We get the 25 articles as expected.

In [20]:
links = doc_tree.xpath('//li[@class="arxiv-result"]/div/p/a')
len(links)

25

In [26]:
first_article_url = links[0].xpath("@href")[0]
first_article_url

'https://arxiv.org/abs/2004.11364'

Get the page corresponding to the article

In [27]:
response2 = requests.get(first_article_url)
response2

<Response [200]>

In [28]:
doc_tree2 = html.fromstring(response2.content)
doc_tree2

<Element html at 0x10fe2cf48>

Search for authors

In [97]:
for element in doc_tree2.xpath('//*[@id="abs"]/div[@class="authors"]//a'):
    print(element.text)
    

Richard Tucker
Noah Snavely


In [98]:
for title in doc_tree2.xpath('//div[@id="abs"]//h1[@class="title mathjax"]//*[text()]'):
    print(title.text)
    


    
#Titre récupérer dans deuxième page, ne fonctionne pas 
#//div[@id="abs"]//*[@class="title mathjax"]//*[text()]

#Titre récupérer dans 1ere page, fonctionne
#doc_tree.xpath('//*[@id="main-container"]//*[@class= "title is-5 mathjax"]')

Title:


In [99]:
date_str = doc_tree2.xpath('//div[@class="dateline"]')[0].text
date_str

'\n  \n  \n  \n    \n  \n  \n    \n    \n  \n\n  [Submitted on 23 Apr 2020]'

In [11]:
re.findall('Submitted on (.*)]', date_str)[0]

'22 Apr 2020'

Search for PDF link

In [12]:
pdf_element = doc_tree2.xpath('//div[@class="full-text"]//a[text()="PDF"]')[0]
pdf_url = pdf_element.xpath("@href")[0]
pdf_url


'/pdf/2004.10746'

In [13]:
response3 = requests.get(f"https://arxiv.org/{pdf_url}")
response3

<Response [200]>

In [14]:
response3.headers['Content-Type']

'application/pdf'

In [15]:
from IPython.display import IFrame
IFrame(f"https://arxiv.org/{pdf_url}", width=600, height=800)